**Instalações necessárias**
- FASE 1: Instalação do python no notebook do colab.

In [ ]:
!pip install -U -q PyDrive

**Importações necessárias**
- FASE 2: Importações das bibliotecas a serem utilizadas.

In [ ]:
#Todos os imports necessarios
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
import plotly.express as px
import pyarrow as pa
import pyarrow.parquet as pq
from statistics import median
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

**Autenticação de segurança**
- FASE 3: Verificação de acesso do email INTELI para acessar a base de dados fornecida pelo Banco Pan. Importante para a segurança dos dados.

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Download dos Dados**
- FASE 4: Realização do download dos dados da base para esse notebook do Colab, para a visualização e manipulação.

In [ ]:
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1nYYM_5xH6bWL5gPqBTx_mt0SCzm4EiuU'})
downloaded.GetContentFile("06120018.zip")
shutil.unpack_archive("06120018.zip", "./dataset")

**Conversão das colunas**
- FASE 5: Converter os tipos das colunas de int/float64 para int/float, dessa maneira, os dados acabam diminuindo o tamanho o seu tamanho.

In [ ]:
def convert_columns(df_filtered):
    # Itera sobre as colunas do DataFrame
    for coluna in df_filtered.columns:
        # Verifica se o tipo de dado da coluna é float64
        if df_filtered[coluna].dtype == 'float64':
            # Converte o tipo de dado da coluna para float16 (meia precisão)
            df_filtered[coluna] = df_filtered[coluna].astype('float16')

        # Verifica se o tipo de dado da coluna é int64
        if df_filtered[coluna].dtype == 'int64':
            # Converte o tipo de dado da coluna para int16
            df_filtered[coluna] = df_filtered[coluna].astype('int16')

    # Retorna o DataFrame com as colunas convertidas
    return df_filtered

**Filtragem das colunas**
- FASE 6: Estruturação e filtragem de todas as colunas de acordo com a indicação realizada pelo parceiro.

In [ ]:
#Filtro para as colunas
filtered_cols = [
    "amscHprsovDrivF-1a",
    "amscHprsovDrivF-1b",
    "amscHprsovDrivF-2b",
    "amscPrsovDrivF-1a",
    "amscPrsovDrivF-1b",
    "amscPrsovDrivF-2b",
    "basBleedLowPressF-1a",
    "basBleedLowPressF-2b",
    "basBleedLowTempF-1a",
    "basBleedLowTempF-2b",
    "basBleedOverPressF-1a",
    "basBleedOverPressF-2b",
    "basBleedOverTempF-1a",
    "basBleedOverTempF-2b",
    "bleedFavTmCmd-1a",
    "bleedFavTmCmd-1b",
    "bleedFavTmCmd-2a",
    "bleedFavTmCmd-2b",
    "bleedFavTmFbk-1a",
    "bleedFavTmFbk-1b",
    "bleedFavTmFbk-2b",
    "bleedHprsovCmdStatus-1a",
    "bleedHprsovCmdStatus-1b",
    "bleedHprsovCmdStatus-2a",
    "bleedHprsovCmdStatus-2b",
    "bleedHprsovOpPosStatus-1a",
    "bleedHprsovOpPosStatus-1b",
    "bleedHprsovOpPosStatus-2a",
    "bleedHprsovOpPosStatus-2b",
    "bleedMonPress-1a",
    "bleedMonPress-1b",
    "bleedMonPress-2a",
    "bleedMonPress-2b",
    "bleedOnStatus-1a",
    "bleedOnStatus-1b",
    "bleedOnStatus-2b",
    "bleedOverpressCas-2a",
    "bleedOverpressCas-2b",
    "bleedPrecoolDiffPress-1a",
    "bleedPrecoolDiffPress-1b",
    "bleedPrecoolDiffPress-2a",
    "bleedPrecoolDiffPress-2b",
    "bleedPrsovClPosStatus-1a",
    "bleedPrsovClPosStatus-2a",
    "bleedPrsovFbk-1a",
    'message0422DAA-1',
    'message0418DAA-1',
    'messageInhibitPhases-1'
]

**Limpeza, aplicação da filtragem, agrupamento e concatenação dos arquivos PARQUET**
- FASE 7: Nesta fase, limpamos os dados tirando os nulos e duplicados, filtramo as colunas de acordo com o filtro criado anteriormente, agrupamos as linhas por faze de voo e calculamos a mediana de cada coluna e por fim a união de todos os arquivos parquet em um só.

In [ ]:
# Inicializa um DataFrame vazio
df = pd.DataFrame()

# Contador para acompanhar o número de DataFrames processados
df_number = 0

# Percorre os arquivos no diretório especificado
for _, _, files in os.walk('./dataset/06120018'):
    for actual_file in files:
        # Lê o arquivo Parquet
        df_read = pd.read_parquet("./dataset/06120018/" + actual_file, engine='auto')

        # Aplica uma função chamada 'convert_columns' ao DataFrame
        df_read = convert_columns(df_read)

        # Remove linhas duplicadas e preenche valores ausentes com 0
        df_read = df_read.drop_duplicates(keep='first').fillna(0)

        # Seleciona colunas específicas usando 'filtered_cols'
        df_read = df_read[filtered_cols]

        # Percorre diferentes valores de índice
        for index in range(8):
            # Filtra as linhas com base em uma condição
            fase_flight = (df_read['messageInhibitPhases-1'] == index)

            # Agrupa e agrega colunas selecionadas com base no índice
            result = df_read[fase_flight].groupby('messageInhibitPhases-1').agg({
            'amscHprsovDrivF-1a': 'mean',
            'amscHprsovDrivF-1b': 'mean',
            'amscHprsovDrivF-2b': 'mean',
            'amscPrsovDrivF-1a': 'mean',
            'amscPrsovDrivF-1b': 'mean',
            'amscPrsovDrivF-2b': 'mean',
            'basBleedLowPressF-1a': 'mean',
            'basBleedLowPressF-2b': 'mean',
            'basBleedLowTempF-1a': 'mean',
            'basBleedLowTempF-2b': 'mean',
            'basBleedOverPressF-1a': 'mean',
            'basBleedOverPressF-2b': 'mean',
            'basBleedOverTempF-1a': 'mean',
            'basBleedOverTempF-2b': 'mean',
            'bleedFavTmCmd-1a': 'mean',
            'bleedFavTmCmd-1b': 'mean',
            'bleedFavTmCmd-2a': 'mean',
            'bleedFavTmCmd-2b': 'mean',
            'bleedFavTmFbk-1a': 'mean',
            'bleedFavTmFbk-1b': 'mean',
            'bleedFavTmFbk-2b': 'mean',
            'bleedHprsovCmdStatus-1a': 'mean',
            'bleedHprsovCmdStatus-1b': 'mean',
            'bleedHprsovCmdStatus-2a': 'mean',
            'bleedHprsovCmdStatus-2b': 'mean',
            'bleedHprsovOpPosStatus-1a': 'mean',
            'bleedHprsovOpPosStatus-1b': 'mean',
            'bleedHprsovOpPosStatus-2a': 'mean',
            'bleedHprsovOpPosStatus-2b': 'mean',
            'bleedMonPress-1a': 'mean',
            'bleedMonPress-1b': 'mean',
            'bleedMonPress-2a': 'mean',
            'bleedMonPress-2b': 'mean',
            'bleedOnStatus-1a': 'mean',
            'bleedOnStatus-1b': 'mean',
            'bleedOnStatus-2b': 'mean',
            'bleedOverpressCas-2a': 'mean',
            'bleedOverpressCas-2b': 'mean',
            'bleedPrecoolDiffPress-1a': 'mean',
            'bleedPrecoolDiffPress-1b': 'mean',
            'bleedPrecoolDiffPress-2a': 'mean',
            'bleedPrecoolDiffPress-2b': 'mean',
            'bleedPrsovClPosStatus-1a': 'mean',
            'bleedPrsovClPosStatus-2a': 'mean',
            'bleedPrsovFbk-1a': 'mean',
            'message0422DAA-1': 'mean',
            'message0418DAA-1': 'mean',
            'messageInhibitPhases-1': 'mean'
        })

            # Concatena o resultado com o DataFrame principal
            df = pd.concat([df, result], axis=0, ignore_index=True)

            # Limpa a variável de resultado
            result = None

        # Incrementa o contador de DataFrames
        df_number = df_number + 1

        # Imprime o número atual do DataFrame
        print(df_number)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


**Exportar em PARQUET**
- FASE 8: Para não perde o que foi processado, exportamos o dataset para um arquivo parquet.

In [ ]:
output_file = 'output.parquet'
df.to_parquet(output_file)

**Importar o novo PARQUET**
- FASE 9: Reimportar o novo parquet gerado

In [ ]:
downloaded = drive.CreateFile({'id': '1tvSj815g-5CBtb04fOQ06LI5_nJ7ihos'})
downloaded.GetContentFile("output.parquet")

df = pd.read_parquet("./output.parquet",engine='auto')

In [ ]:
df.head()

,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,amscPrsovDrivF-2b,basBleedLowPressF-1a,basBleedLowPressF-2b,basBleedLowTempF-1a,basBleedLowTempF-2b,...,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,message0422DAA-1,message0418DAA-1,messageInhibitPhases-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.26904,0.270028,0.259412,0.259429,0.000018,0.000018,85.745446,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.184620,0.520498,0.000000,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.327721,0.327431,0.000000,0.0,0.0,2.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.0


**Dropar colunas**
- FASE 10: Nesta fase, após uma análise, podemos constatar que apesar do tratamento, ainda sobraram varias colunas com informações vazias. Dessa forma, mapeamos essas colunas e dropamos elas.

In [ ]:
df.drop([
    'amscHprsovDrivF-1a',
    'amscHprsovDrivF-1b',
    'amscHprsovDrivF-2b',
    'amscPrsovDrivF-1a',
    'amscPrsovDrivF-1b',
    'amscPrsovDrivF-2b',
    'basBleedLowPressF-1a',
    'basBleedLowPressF-2b',
    'basBleedLowTempF-1a',
    'basBleedLowTempF-2b',
    'basBleedOverPressF-1a',
    'basBleedOverPressF-2b',
    'basBleedOverTempF-1a',
    'basBleedOverTempF-2b',
    'bleedOverpressCas-2a',
    'bleedOverpressCas-2b',
], axis=1, inplace=True)

In [ ]:
df.head()

,bleedFavTmCmd-1a,bleedFavTmCmd-1b,bleedFavTmCmd-2a,bleedFavTmCmd-2b,bleedFavTmFbk-1a,bleedFavTmFbk-1b,bleedFavTmFbk-2b,bleedHprsovCmdStatus-1a,bleedHprsovCmdStatus-1b,bleedHprsovCmdStatus-2a,...,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,message0422DAA-1,message0418DAA-1,messageInhibitPhases-1
0,48.403783,3.133611,45.600019,45.602387,49.758077,3.45565,47.141908,0.000000,0.000000,0.000000,...,0.26904,0.270028,0.259412,0.259429,0.000018,0.000018,85.745446,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.815098,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.184620,0.520498,0.000000,0.0,0.0,1.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.661829,0.000000,0.662409,...,0.00000,0.000000,0.000000,0.000000,0.327721,0.327431,0.000000,0.0,0.0,2.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.0


**Conversão das colunas**
- FASE 11: Embora tenhamos tratado as colunas anteriormente, por padrão o formato de arquivo parquet vai definir as colunas em formato 64, mas para dimunuir o tamanho ocupado pelo arquivo, convertemos novamente para Float16.

In [ ]:
df = convert_columns(df)

In [ ]:
df.dtypes

bleedFavTmCmd-1a             float16
bleedFavTmCmd-1b             float16
bleedFavTmCmd-2a             float16
bleedFavTmCmd-2b             float16
bleedFavTmFbk-1a             float16
bleedFavTmFbk-1b             float16
bleedFavTmFbk-2b             float16
bleedHprsovCmdStatus-1a      float16
bleedHprsovCmdStatus-1b      float16
bleedHprsovCmdStatus-2a      float16
bleedHprsovCmdStatus-2b      float16
bleedHprsovOpPosStatus-1a    float16
bleedHprsovOpPosStatus-1b    float16
bleedHprsovOpPosStatus-2a    float16
bleedHprsovOpPosStatus-2b    float16
bleedMonPress-1a             float16
bleedMonPress-1b             float16
bleedMonPress-2a             float16
bleedMonPress-2b             float16
bleedOnStatus-1a             float16
bleedOnStatus-1b             float16
bleedOnStatus-2b             float16
bleedPrecoolDiffPress-1a     float16
bleedPrecoolDiffPress-1b     float16
bleedPrecoolDiffPress-2a     float16
bleedPrecoolDiffPress-2b     float16
bleedPrsovClPosStatus-1a     float16
b

**Renomear as colunas**
- FASE 12: Para facilitar na hora de utilizar as colunas na fase de plotagem de gráficos, iremos substituir os hífens por sublinhados.

In [ ]:
# Substitui todos os hífens '-' por underscores '_' nos nomes das colunas do DataFrame
df.columns = df.columns.str.replace('-', '_')

**Verficação das correlações entre as colunas**
- FASE 12: Para auxiliar no processo de separação das colunas para treinamento, iremos analisar as relações entre as colunas a fim de obter alguma percepção.

In [ ]:
# Calcula a matriz de correlação para o DataFrame df e aplica um estilo de gradiente de cores
matriz_corr = df.corr().style.background_gradient(cmap='coolwarm')

# Mostra a matriz de correlação com o estilo de gradiente aplicado
matriz_corr

,bleedFavTmCmd_1a,bleedFavTmCmd_1b,bleedFavTmCmd_2a,bleedFavTmCmd_2b,bleedFavTmFbk_1a,bleedFavTmFbk_1b,bleedFavTmFbk_2b,bleedHprsovCmdStatus_1a,bleedHprsovCmdStatus_1b,bleedHprsovCmdStatus_2a,bleedHprsovCmdStatus_2b,bleedHprsovOpPosStatus_1a,bleedHprsovOpPosStatus_1b,bleedHprsovOpPosStatus_2a,bleedHprsovOpPosStatus_2b,bleedMonPress_1a,bleedMonPress_1b,bleedMonPress_2a,bleedMonPress_2b,bleedOnStatus_1a,bleedOnStatus_1b,bleedOnStatus_2b,bleedPrecoolDiffPress_1a,bleedPrecoolDiffPress_1b,bleedPrecoolDiffPress_2a,bleedPrecoolDiffPress_2b,bleedPrsovClPosStatus_1a,bleedPrsovClPosStatus_2a,bleedPrsovFbk_1a,message0422DAA_1,message0418DAA_1,messageInhibitPhases_1
bleedFavTmCmd_1a,1.000000,-0.057916,0.907642,0.908195,0.999982,-0.053673,0.910992,-0.142504,-0.142177,-0.142307,-0.142308,-0.247247,-0.247285,-0.247631,-0.247690,0.655557,0.655555,0.586380,0.586609,-0.430975,-0.430973,-0.423091,0.672195,0.672274,0.071711,0.072147,-0.149009,-0.158621,0.994912,-0.004829,0.002983,-0.385534
bleedFavTmCmd_1b,-0.057916,1.000000,-0.059189,-0.059250,-0.055652,0.999970,-0.057570,-0.142221,-0.141896,-0.142025,-0.142026,-0.246757,-0.246795,-0.247140,-0.247199,0.656117,0.656109,0.584897,0.585134,-0.430121,-0.430118,-0.422252,0.670620,0.670666,0.070509,0.070951,-0.148714,-0.158307,-0.056728,0.009009,-0.001670,-0.384770
bleedFavTmCmd_2a,0.907642,-0.059189,1.000000,0.999947,0.908462,-0.055047,0.999874,-0.138909,-0.138591,-0.138717,-0.138718,-0.241010,-0.241047,-0.241384,-0.241441,0.625504,0.625505,0.683794,0.683437,-0.420103,-0.420101,-0.412418,0.590610,0.590812,0.362526,0.362802,-0.145250,-0.154620,0.907723,-0.004719,0.002946,-0.375808
bleedFavTmCmd_2b,0.908195,-0.059250,0.999947,1.000000,0.909016,-0.055101,0.999947,-0.139055,-0.138736,-0.138863,-0.138863,-0.241263,-0.241300,-0.241637,-0.241694,0.626012,0.626013,0.683701,0.683452,-0.420543,-0.420541,-0.412850,0.590966,0.591169,0.360452,0.361084,-0.145403,-0.154782,0.908275,-0.004724,0.002947,-0.376203
bleedFavTmFbk_1a,0.999982,-0.055652,0.908462,0.909016,1.000000,-0.051393,0.911833,-0.143200,-0.142871,-0.143002,-0.143002,-0.248455,-0.248492,-0.248840,-0.248899,0.658052,0.658051,0.589123,0.589355,-0.433079,-0.433077,-0.425156,0.673774,0.673856,0.071512,0.071953,-0.149736,-0.159395,0.994986,-0.004681,0.003340,-0.387416
bleedFavTmFbk_1b,-0.053673,0.999970,-0.055047,-0.055101,-0.051393,1.000000,-0.053397,-0.143363,-0.143034,-0.143165,-0.143165,-0.248738,-0.248775,-0.249123,-0.249182,0.660442,0.660434,0.589593,0.589837,-0.433572,-0.433570,-0.425641,0.673640,0.673690,0.070841,0.071302,-0.149907,-0.159577,-0.052492,0.009944,-0.001520,-0.387858
bleedFavTmFbk_2b,0.910992,-0.057570,0.999874,0.999947,0.911833,-0.053397,1.000000,-0.140008,-0.139687,-0.139814,-0.139815,-0.242917,-0.242954,-0.243293,-0.243351,0.629669,0.629670,0.684647,0.684416,-0.423426,-0.423424,-0.415680,0.594309,0.594514,0.351685,0.352317,-0.146399,-0.155842,0.911059,-0.004588,0.003008,-0.378781
bleedHprsovCmdStatus_1a,-0.142504,-0.142221,-0.138909,-0.139055,-0.143200,-0.143363,-0.140008,1.000000,-0.292124,0.942090,0.942089,0.595215,0.595206,0.544626,0.544617,-0.221759,-0.221763,-0.210232,-0.210381,0.241119,0.241125,0.167272,-0.204641,-0.204719,-0.017611,-0.017940,-0.060929,0.039408,-0.142447,-0.010477,-0.005091,0.162867
bleedHprsovCmdStatus_1b,-0.142177,-0.141896,-0.138591,-0.138736,-0.142871,-0.143034,-0.139687,-0.292124,1.000000,-0.292036,-0.292038,0.591046,0.591036,0.536909,0.536902,-0.221251,-0.221255,-0.209751,-0.209899,0.237944,0.237930,0.160461,-0.204172,-0.204250,-0.017571,-0.017899,-0.057255,0.047045,-0.142121,-0.010453,-0.005079,0.162181
bleedHprsovCmdStatus_2a,-0.142307,-0.142025,-0.138717,-0.138863,-0.143002,-0.143165,-0.139814,0.942090,-0.292036,1.000000,1.000000,0.546816,0.546798,0.595807,0.595795,-0.221452,-0.221456,-0.209942,-0.210091,0.202055,0.202058,0.226599,-0.204358,-0.204436,-0.017587,-0.017915,-0.009160,-0.038778,-0.142250,-0.010463,-0.005084,0.235721


In [ ]:
target_corr = matriz_corr.data.message0418DAA_1
df_corr = target_corr.abs().sort_values(ascending=False).to_frame().style.background_gradient(cmap='coolwarm')
df_corr

,message0418DAA_1
message0418DAA_1,1.000000
bleedOnStatus_1a,0.015396
bleedOnStatus_1b,0.015396
bleedOnStatus_2b,0.015115
messageInhibitPhases_1,0.013773
bleedMonPress_2b,0.012735
bleedMonPress_2a,0.012731
bleedHprsovOpPosStatus_2b,0.008849
bleedHprsovOpPosStatus_2a,0.008846
bleedHprsovOpPosStatus_1b,0.008834


In [ ]:
target_corr = matriz_corr.data.message0422DAA_1
df_corr = target_corr.abs().sort_values(ascending=False).to_frame().style.background_gradient(cmap='coolwarm')
df_corr

,message0422DAA_1
message0422DAA_1,1.000000
bleedOnStatus_1a,0.031687
bleedOnStatus_1b,0.031687
bleedOnStatus_2b,0.031107
bleedMonPress_1b,0.030155
bleedMonPress_1a,0.030149
messageInhibitPhases_1,0.028346
bleedHprsovOpPosStatus_2b,0.018211
bleedHprsovOpPosStatus_2a,0.018207
bleedHprsovOpPosStatus_1b,0.018181


**Preparação para treino**
- FASE 13: Separa as colunas no sentido, quais seram para identificação e quais de definição.

In [ ]:
columns_predication = [
    'bleedFavTmCmd_1a',
    'bleedFavTmCmd_1b',
    'bleedFavTmCmd_2a',
    'bleedFavTmCmd_2b',
    'bleedFavTmFbk_1a',
    'bleedFavTmFbk_1b',
    'bleedFavTmFbk_2b',
    'bleedHprsovCmdStatus_1a',
    'bleedHprsovCmdStatus_1b',
    'bleedHprsovCmdStatus_2a',
    'bleedHprsovCmdStatus_2b',
    'bleedHprsovOpPosStatus_1a',
    'bleedHprsovOpPosStatus_1b',
    'bleedHprsovOpPosStatus_2a',
    'bleedHprsovOpPosStatus_2b',
    'bleedMonPress_1a',
    'bleedMonPress_1b',
    'bleedMonPress_2a',
    'bleedMonPress_2b',
    'bleedOnStatus_1a',
    'bleedOnStatus_1b',
    'bleedOnStatus_2b',
    'bleedPrecoolDiffPress_1a',
    'bleedPrecoolDiffPress_1b',
    'bleedPrecoolDiffPress_2a',
    'bleedPrecoolDiffPress_2b',
    'bleedPrsovClPosStatus_1a',
    'bleedPrsovClPosStatus_2a',
    'bleedPrsovFbk_1a',
    'messageInhibitPhases_1']

In [ ]:
colum_classification = ['message0422DAA_1',	'message0418DAA_1']

In [ ]:
x = df[columns_predication]
y = df[colum_classification]